## Part III:  Loading Evaluation Labels

In [1]:
# If necessary:
import os
os.remove('snorkel.db');
os.system('cp snorkel.db\ candidates snorkel.db');

In [2]:
%load_ext autoreload
%autoreload 2

from snorkel import SnorkelSession
session = SnorkelSession()

In [3]:
from snorkel.models import candidate_subclass

Part_Temp = candidate_subclass('Part_Temp', ['part','temp'])

## Loading the Training `CandidateSet`

In [4]:
from snorkel.models import CandidateSet
candidates = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
print "%s contains %d Candidates" % (candidates, len(candidates))

Candidate Set (Hardware Training Candidates) contains 11549 Candidates


In [5]:
from snorkel.loaders import ExternalAnnotationsLoader

loader = ExternalAnnotationsLoader(session, Part_Temp, 
                                   'Hardware Training Candidates -- Gold',
                                   'Hardware Training Labels -- Gold',
                                   expand_candidate_set=True)

In [6]:
import os
from hardware_utils import load_hardware_labels

filename = os.environ['SNORKELHOME'] + '/tutorials/tables/data/hardware/hardware_gold.csv'
%time load_hardware_labels(loader, candidates, filename, ['part','temp'], gold_attrib='stg_temp_min')

[========================================] 100%

CPU times: user 5min 38s, sys: 21.5 s, total: 6min
Wall time: 6min 5s


In [7]:
from snorkel.models import Label

train = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates').one()
train_gold = session.query(CandidateSet).filter(
    CandidateSet.name == 'Hardware Training Candidates -- Gold').one()
print "%d/%d Candidates have positive Labels" % (len(train_gold), len(train))
print "%d Labels loaded" % session.query(Label).filter(
    Label.key == loader.annotation_key).count()

2813/11549 Candidates have positive Labels
2813 Labels loaded


In [9]:
for set_name in ['Development']:
    candidate_set_name = 'Hardware %s Candidates' % set_name
    candidates = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name).one()
    loader = ExternalAnnotationsLoader(session, Part_Temp, 
                                       'Hardware %s Candidates -- Gold' % set_name,
                                       'Hardware %s Labels -- Gold' % set_name,
                                       expand_candidate_set=True)
    %time load_hardware_labels(loader, candidates, filename, ['part','temp'], gold_attrib='stg_temp_min')
    candidates_gold = session.query(CandidateSet).filter(
        CandidateSet.name == candidate_set_name + ' -- Gold').one()
    print "%d/%d Candidates in %s have positive Labels" % (
        len(candidates_gold), len(candidates), candidates)
    print "%d Labels loaded" % session.query(Label).filter(
        Label.key == loader.annotation_key).count()

[========================================] 100%

CPU times: user 29.6 s, sys: 2.89 s, total: 32.5 s
Wall time: 32.7 s
362/3163 Candidates in Candidate Set (Hardware Development Candidates) have positive Labels
362 Labels loaded


In [10]:
# If necessary
import os
os.system('cp snorkel.db snorkel.db\ labels');

Next, in Part 4, we will auto-generate features for the `Candidates`